In [1]:
import pandas as pd
import numpy as np

# Carregando os dados

In [2]:
# para os users
df_users = pd.read_parquet('../data/DatasetsLimpos/yelp_academic_dataset_user.parquet')

# para os reviews
df_reviews = pd.read_parquet('../data/DatasetsLimpos/yelp_academic_dataset_review.parquet')

In [3]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   review_count        float32
 2   useful              float32
 3   funny               float32
 4   cool                float32
 5   fans                float32
 6   compliment_hot      float32
 7   compliment_more     float32
 8   compliment_profile  float32
 9   compliment_cute     float32
 10  compliment_list     float32
 11  compliment_note     float32
 12  compliment_plain    float32
 13  compliment_cool     float32
 14  compliment_funny    float32
 15  compliment_writer   float32
 16  compliment_photos   float32
 17  account_age         int64  
 18  chato               float32
dtypes: float32(17), int64(1), object(1)
memory usage: 159.2+ MB


In [4]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      object
 1   business_id  object
 2   stars        uint8 
 3   useful       int32 
 4   funny        int32 
 5   cool         int32 
 6   text         object
dtypes: int32(3), object(3), uint8(1)
memory usage: 246.7+ MB


In [5]:
df_users.shape, df_reviews.shape

((1987897, 19), (6990280, 7))

In [6]:
df_reviews.head()

,user_id,business_id,stars,useful,funny,cool,text
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,1,1,1,"If you decide to eat here, just be aware it is..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,1,1,I've taken a lot of spin classes over the year...
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,1,1,1,Family diner. Had the buffet. Eclectic assortm...
3,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,1,1,"Wow! Yummy, different, delicious. Our favo..."
4,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,1,1,Cute interior and owner (?) gave us tour of up...


In [7]:
df_users.head()

,user_id,review_count,useful,funny,cool,fans,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,account_age,chato
0,qVc8ODYU5SZjKXVBgXdI7w,6.376248,8.884571,7.140230,8.698968,5.597378,5.532275,4.215356,4.055574,4.072751,3.031017,5.458386,6.741368,6.152133,6.152133,5.487773,5.207945,5864,0.992327
1,j14WgRoU_-2ZE1aw1dXrJg,8.374642,10.671132,9.477977,10.214046,8.052207,7.045531,5.586193,5.229601,5.073411,5.536224,7.522789,8.861735,8.049975,8.049975,7.328909,7.574927,5133,1.037433
2,2WnXYQFK0hXEoTxPtV2zvg,6.503866,7.644306,6.920393,6.913457,4.002198,4.518722,2.754824,2.543041,2.981546,1.743668,4.230015,4.592270,4.801709,4.801709,3.630145,3.031017,5317,1.168675
3,SZDeASXq7o05mMNLshsdIA,5.423708,6.243620,5.807296,5.709494,3.424858,3.285348,1.904832,1.313262,2.165422,1.551445,2.689090,2.929501,3.357534,3.357534,2.543041,2.461150,6286,0.908665
4,hA5lMy-EnncsH4JoR-hFGQ,4.403278,3.456893,2.874597,2.274009,1.313262,1.313262,1.313262,1.000000,1.000000,1.000000,1.313262,1.313262,1.000000,1.000000,1.000000,1.000000,5884,1.096045


# Extraindo embeddings dos reviews - text

Para essa task, devemos extrair os embeddinggs de texto dos reviews e estipular uma média para cada usuário.

OBS: do dataset review, só aproveitamos o texto e o id do usuário.

In [8]:
# relembrando quantos usuarios tem reviews
df_reviews[df_reviews['text'].notna()].user_id.nunique()

1987929

In [9]:
df_users.user_id.nunique()

1987897

In [10]:
# agrupando os textos por usuário

In [11]:
df_reviews_text = df_reviews[['user_id', 'text']].groupby('user_id').agg(lambda x: list(x))

In [12]:
df_reviews_text.shape

(1987929, 1)

> O dataframe contém, por usuário, uma lista de reviews (textos).

## Extraindo os embeddings

In [13]:
import torch
from transformers import BertTokenizer, BertModel
import time
from multiprocessing import  Pool


# Load pretrained model/tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True).to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
def get_bert_embedding(text_list):
    # if text_list is over than 15 items, split it in groups of 15
    # print(len(text_list))
    if len(text_list) > 15:
        # print("Text list is over than 15 items, split it in groups of 15")
        text_list = [text_list[i:i + 15] for i in range(0, len(text_list), 15)]
    else:
        text_list = [text_list]
    embs = []
    for text in text_list:
        start = time.time()
        tokens = []
        # clip text if it is too long (more than 512 tokens)
        for idx in range(len(text)):
            if len(text[idx]) > 512:
                text[idx] = text[idx][:510]
            # Add the special tokens.
            marked_text = "[CLS] " + text[idx] + " [SEP]"
            # Split the sentence into tokens.
            tokenized_text = tokenizer.tokenize(marked_text)
            # padding if text is less than 512 tokens
            if len(tokenized_text) < 512:
                tokenized_text = tokenized_text + ["[PAD]"] * (512 - len(tokenized_text))
            # Map the token strings to their vocabulary indexes.
            indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
            tokens.append(indexed_tokens)
        # Convert inputs to PyTorch tensors and put them on GPU
        tokens_tensor = torch.tensor(tokens).to('cuda')

        # Put the model in "evaluation" mode,meaning feed-forward operation.
        model.eval()
        # Run the text through BERT, and collect all the hidden states produced from all 12 layers.
        with torch.no_grad():
            outputs = model(tokens_tensor)[2][-4:]

        # sum of last four layer
        outputs = torch.stack(outputs, dim=1)
        outputs = outputs.sum(1)
        # mean of the tokens, results in one vector of 768 dimensions per text
        outputs = torch.mean(outputs, 1).squeeze(0).cpu().numpy()
        print("Time to get embedding: ", time.time() - start)
        embs.append(outputs)
    return embs

In [ ]:
# extraindo os embeddings
df_reviews_text['embeddings'] = df_reviews_text['text'].apply(get_bert_embedding)

Time to get embedding:  0.09251761436462402
Time to get embedding:  0.049523353576660156
Time to get embedding:  0.047005414962768555
Time to get embedding:  0.07351374626159668
Time to get embedding:  0.05051612854003906
Time to get embedding:  0.11351823806762695
Time to get embedding:  0.026000499725341797
Time to get embedding:  0.09451460838317871
Time to get embedding:  0.027004003524780273
Time to get embedding:  0.07251334190368652
Time to get embedding:  0.22604703903198242
Time to get embedding:  0.1390388011932373
Time to get embedding:  0.02699565887451172
Time to get embedding:  0.025999784469604492
Time to get embedding:  0.02400517463684082
Time to get embedding:  0.07251214981079102
Time to get embedding:  0.07210922241210938
Time to get embedding:  0.028000354766845703
Time to get embedding:  0.09351706504821777
Time to get embedding:  0.02700042724609375
Time to get embedding:  0.026514768600463867
Time to get embedding:  0.025999784469604492
Time to get embedding:  0

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# salvando o dataframe
df_reviews_text.to_parquet('../data/DatasetsLimpos/review_text.parquet')